## Installation

In [ ]:
pip install wandb numpy pandas matplotlib

## Q1: fashion-MNIST dataset

In [10]:
import wandb
from keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

classes = {
    0: "T-shirt/top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot"
}

def logClassImages(project_name:str):
  wandb.init(project=project_name)
  wandb_image_indices = []

  for classNumber in range(10):
    for j in range(len(y_test)):
      if y_test[j] == classNumber:
        wandb_image_indices.append(x_test[j])
        break

  wandb_images = [wandb.Image(wandb_image_indices[i], caption = classes[i]) for i in range(10)]
  wandb.log({"Sample images for each class": wandb_images})
  wandb.finish()

# logClassImages("da6401_assignment1")

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Feedforward neural network



In [3]:
import wandb
from keras.datasets import fashion_mnist
import numpy as np

In [30]:
# Activation functions:
def softmax(x):
  pass

In [32]:
class FeedForwardNeuralNetwork:
  # class variables
  optimizersMap = {"a":lambda x: x+1}
  lossFunctionsMap = {"a":lambda x: x+1}
  activationFunctionsMap = {"a":lambda x: x+1}

  def __init__(self,
               input_size=784, output_size=10,
               n_hiddenLayers=3, n_neuronsPerLayer=32,
               activationFun="sigmoid",
               weight_init="random",
               batch_size=64,
               lossFunc="cross_entropy",
               optimizer="adam",
               learning_rate=0.001,
               momentum=0.5,
               beta=0.9, beta1=0.9, beta2=0.99,
               epsilon=1e-8, weight_decay=0.01,
               epochs=10):

    # Inialtization parameters
    self.input_size = input_size  # no of features
    self.output_size = output_size
    self.n_hiddenLayers = n_hiddenLayers
    self.n_neuronsPerLayer = n_neuronsPerLayer
    self.weight_init = weight_init
    self.epochs = epochs

    self.activationFun = FeedForwardNeuralNetwork.activationFunctionsMap[activationFun]
    self.lossFunc = FeedForwardNeuralNetwork.lossFunctionsMap[lossFunc]
    self.optimizer = FeedForwardNeuralNetwork.optimizersMap[optimizer]

    # paramters required for optimizers
    self.batch_size = batch_size
    self.learning_rate = learning_rate
    self.momentum = momentum
    self.beta = beta
    self.beta1 = beta1
    self.beta2 = beta2
    self.epsilon = epsilon
    self.weight_decay = weight_decay

    # weights and biases matrices
    self.weights = []
    self.biases = []
    self.initializeWeightsAndBiases()

    # pre-activation(a) and post-activation(h) values
    self.a = []
    self.h = []

  '''
    Weights,Biases initialization based on weight_init parameter

    weights[0]: input layer to first hidden layer  : input_size x n_neuronsPerLayer
    weights[1]: first hidden layer to second hidden layer : n_neuronsPerLayer x n_neuronsPerLayer
    ...
    weights[n_hiddenLayers]: last hidden layer to output layer : n_neuronsPerLayer x output_size

    biases[i] : bias for ith layer : 1 x n_neuronsPerLayer   (i:0 to n_hiddenLayers-1)
    biases[n_hiddenLayers]: 1 x output_size
  '''
  def initializeWeightsAndBiases(self):
    # biases for both types
    for _ in range(self.n_hiddenLayers):
      self.biases.append(np.zeros(self.n_neuronsPerLayer))
    self.biases.append(np.zeros(self.output_size))   # biases[n_hiddenLayers]

    if(self.weight_init == "random"):   # Random Normal
      # weights[0]
      self.weights.append(np.random.randn(self.input_size, self.n_neuronsPerLayer))

      # weights[1] -> weights[n_hiddenLayers-1]
      for _ in range(self.n_hiddenLayers-1):
        self.weights.append(np.random.randn(self.n_neuronsPerLayer, self.n_neuronsPerLayer))

      # weights[n_hiddenLayers]
      self.weights.append(np.random.randn(self.n_neuronsPerLayer, self.output_size))
    elif(self.weight_init == "Xavier"):   # Xavier Normal: mean = 0, variance = 2/(n_input + n_output)
      # weights[0]
      self.weights.append(np.random.normal(loc=0.0, scale=np.sqrt(2/(self.input_size + self.n_neuronsPerLayer)), size=(self.input_size, self.n_neuronsPerLayer)))

      for _ in range(self.n_hiddenLayers-1):
        self.weights.append(np.random.normal(loc=0.0, scale=np.sqrt(2/(self.n_neuronsPerLayer + self.n_neuronsPerLayer)), size=(self.n_neuronsPerLayer, self.n_neuronsPerLayer)))

      self.weights.append(np.random.normal(loc=0.0, scale=np.sqrt(2/(self.n_neuronsPerLayer + self.output_size)), size=(self.n_neuronsPerLayer, self.output_size)))

  '''
    Forward propagation through the neural network. (for batch)
    Instead of doing one input at a time, this function handles it for a batch using respective sized matrices

    x_batch: B x n where B - batch size, n- no of features = input_size
    x_batch is assumbed to be numpy array when given as input
  '''
  def forwardPropagation(self, x_batch):
    a_pre_activation = []
    h_post_activation = []

    # considering a0,h0 as X values as a1: first layer  (it is calculated from x values)
    a_pre_activation.append(x_batch)
    h_post_activation.append(x_batch)

    # Except last layer since activation function could be different
    for i in range(self.n_hiddenLayers):
      # ai: B x n_neuronsPerLayer, biases[i]: 1 x n_neuronsPerLayer (it will be broadcasted while adding)
      ai = np.matmul(h_post_activation[-1], self.weights[i]) + self.biases[i]
      hi = self.activationFun(ai)

      a_pre_activation.append(ai)
      h_post_activation.append(hi)

    # aL: last layer (activation function is softmax)
    aL = np.matmul(h_post_activation[-1], self.weights[self.n_hiddenLayers]) + self.biases[self.n_hiddenLayers]
    hL = softmax(aL)

    a_pre_activation.append(aL)
    h_post_activation.append(hL)

    return a_pre_activation, h_post_activation

In [27]:
# import numpy as np

# # Create a column vector (shape: (3,1))
# v1 = np.array([[1,2], [2,3], [3,4]])  # Shape (3,1)
# v2 = np.array([[4], [5], [6]])  # Shape (3,1)

# # Append to a list
# vector_list = []
# vector_list.append(v1)
# vector_list.append(v2)

# print(vector_list)

# # flatten each element of x_train, after flattening store it as np array
# x_ftrain = np.array([x.flatten() for x in x_train])
# x1 = [x.flatten() for x in x_train]
# # print(x_ftrain[0].shape)
# # print(x_ftrain.shape)
# # print(x_ftrain[:20].shape)
# # print(y_train[:20].shape)
# # print(x_test.shape)
# # print(y_test.shape)
# m = np.array(np.zeros((784,3)))
# y = np.matmul(x_ftrain[:20],m)
# print(y.shape)

[array([[1, 2],
       [2, 3],
       [3, 4]]), array([[4],
       [5],
       [6]])]
(20, 3)
